In [41]:
#setup - rememeber to switch to tensorflow 2.3 kernel...
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime
import trimesh

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.__version__)

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'
%matplotlib notebook

%load_ext tensorboard

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

2.3.0
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [42]:
#load OFF file from ModelNet10 dir
fn = 'C:/Users/Derm/Desktop/big/ModelNet10/toilet/train/toilet_0069.off'
M = trimesh.load(fn)
test = trimesh.sample.sample_surface(M, 10000)

In [43]:
#use Vedo to plot OG and subsampled surfaces
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp = []

# disp.append(Points(M.vertices, c = 'blue', r = 4))
disp.append(Points(test[0], c = 'red', r = 2))

plt.show(disp, "surface sampling test")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [154]:
#generate toy dataset using all of the toilets in the ModelNet10 repository
numMeshes = 344 #344
ptsPerCloud = 25 
iterPerMesh = 1  #number of times to sample clouds from each mesh

#init vector to store sampled point clouds
x = np.zeros([numMeshes*iterPerMesh, ptsPerCloud*2, 3])

#init vector to store transformations 
y = np.zeros([numMeshes*iterPerMesh, 6])

for i in range(numMeshes):
    print(i)
    fn = 'C:/Users/Derm/Desktop/big/ModelNet10/toilet/train/toilet_%04d.off' %(i+1)
    M = trimesh.load(fn)
    
    sam1 = trimesh.sample.sample_surface(M, ptsPerCloud)[0] #get keyframe scan
    x[i, :ptsPerCloud, :] = sam1

    sam2 = trimesh.sample.sample_surface(M, ptsPerCloud)[0] #get new scan
    angs = 0.2*tf.random.normal([3])
    rot = R_tf(angs)
    trans = 0.5*tf.random.normal([3])
    sam2 = trans + sam2.dot(rot.numpy()) #transform scan
    x[i, ptsPerCloud:, :] = sam2
    
    #save transformation as y
    y[i,:3] = trans.numpy()
    y[i,3:] = angs.numpy()
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [159]:
# print(x[8])
# print(y)
print(np.shape(x))
# print(tf.constant([0.001, 0.01, 0.1])*tf.random.uniform([3]))

(344, 50, 3)


In [121]:
def R_tf(angs):
    """generates rotation matrix using euler angles
    angs = tf.constant(phi, theta, psi) (aka rot about (x,y,z))
            can be single set of angles or batch for multiple cells
    """

    if len(tf.shape(angs)) == 1:
        angs = angs[None,:]

    phi = angs[:,0]
    theta = angs[:,1]
    psi = angs[:,2]

    mat = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]
                        ])

    mat = tf.transpose(mat, [2, 0, 1])
    mat = tf.squeeze(mat)
    return mat

In [161]:
FN = 'C:/Users/Derm/Desktop/big/ModelNet10/toilet/train/25PtsEach'
np.save(FN, x)

In [165]:
test = np.load(FN+'.npy')
print(np.shape(test))

(344, 50, 3)
